<a href="https://colab.research.google.com/github/A00827038/Data_Structures/blob/main/AI_Actividad_Integradora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI. Actividad Integradora

En esta actividad modelaremos el comportamiento de agentes reactivos simples–robots de robots y cajas. Donde el objetivo es que los robots ordenen un almacen con K número de cajas para que todas las cajas terminen en pilas ordenadas de cinco. 


## Reglas

Dado
1. Habitación de MxN espacios.
2. Número de cajas.
3. 5 robots
5. Tiempo máximo de ejecución.

Realizar lo Siguiente
1. Inicializa las posiciones iniciales de las K cajas. Todas las cajas están a nivel de piso, es decir, no hay pilas de cajas.
2. Todos los agentes empiezan en posición aleatorias vacías.
3. Los robots recorreran el grid, agarran una caja de una celda donde sólo haya una caja, y la llevan a una celda donde haya 1 <= caja < 5.

## Imports

Antes de empezar a crear el modelo del juego de la vida con multiagentes es necesario tener instalado los siguientes paquetes:
- `python`: asegúrense de usar la versión 3+.
- `mesa`: el framework de Python para el modelado de agentes.
- `numpy`: es una biblioteca de Python para el manejo de matrices, arreglos, manipulación matemática, lógica y mucho más.
- `matplotlib`: es una biblioteca para crear visualizaciones estáticas, animadas e interactivas en Python.

Para poder modelar el juego de la vida usando el framework de `mesa` es necesario importar dos clases: una para el modelo general, y otro para los agentes. 

In [16]:
!pip3 install mesa

In [17]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `SingleGrid` que fuerza un solo objeto por celda.
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

## Crear el modelo

Antes que nada el presente modelo se encuentra basado en el [tutorial introductorio](https://mesa.readthedocs.io/en/master/tutorials/intro_tutorial.html). Lo modifiqué un poco para que funcionara para el presente problema pero en esencia es lo mismo.

In [18]:
def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid. 
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
          if isinstance(content, robotAgent):
            grid[x][y] = 6
          else:
            grid[x][y] = content.estado

    return grid


class robotAgent(Agent):
    '''
    Un robot que se mueve y limpia el piso.
    '''
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.nextPosition = None;
        self.caja = 0; #Si el robot lleva caja entonces 1, si no, entonces 0
        self.nextCaja = None;
        

    def step(self):
      '''
      Método que checa si hay caja en su celda. Si hay caja entonces se la lleva, y si no, 
      entonces se mueve a una celda adjacente. 
      '''
      #Agente Piso en la Celda donde está el Robot
      this_cell = self.model.grid.get_cell_list_contents([self.pos])
      cajas = [obj for obj in this_cell if isinstance(obj, floorAgent)]
      
      #Possibles lugares a moverse
      possible_steps = self.model.grid.get_neighborhood(
        self.pos,
        moore=False,
        include_center=False)
      
      for neighbour_cell in possible_steps:
        robots = [obj for obj in self.model.grid.get_cell_list_contents([neighbour_cell]) if isinstance(obj, floorAgent)][0]
        if robots.ocupado:
          possible_steps.remove(neighbour_cell)
      
      self.nextPosition = self.pos;
      #Este piso ya no estará ocupado porque el robot se moverá
      floor = [obj for obj in this_cell if isinstance(obj, floorAgent)]
      floor[0].ocupado = False
      self.nextCaja = self.caja; 

      if self.caja == 0 and cajas[0].estado == 1 and (not cajas[0].stack):
        self.nextCaja = 1
        cajas[0].estado = 0
      elif self.caja == 1:
        if cajas[0].stack and (cajas[0].residuo == False and cajas[0].estado < 5 or  cajas[0].residuo == True and cajas[0].estado < self.model.num_cajas%5):
          self.nextCaja = 0
          cajas[0].estado += 1
        else:
          self.nextPosition = self.random.choice(possible_steps)   
      else:
        self.nextPosition = self.random.choice(possible_steps)
          

    def advance(self):
      self.caja = self.nextCaja;
      self.model.grid.move_agent(self, self.nextPosition)
       
      
class floorAgent(Agent):
    '''
    Representa a una celda de tipo piso.
    '''
    def __init__(self, unique_id, model, estado, stack, ocupado, residuo):
        super().__init__(unique_id, model)
        self.estado = estado
        self.stack = stack
        self.ocupado = ocupado
        self.residuo = residuo
       
      

            
class CleanWarehouseModel(Model):
    '''
    Define el modelo de limpia piso con robots.
    '''
    def __init__(self, width, height, K):
        self.num_robots = 5
        self.num_cajas = K
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)
        estados = [0] * width * height
        estadosIndice = list(range(width * height))

        for n in range (self.num_cajas):
          #Escoge un valor random del arreglo de estadosIndice
          #Usamos el valor como indice en el arreglo de estados y le asignamos un valor de 1, indicando que ahí va haber una caja
          #Quitamos el valor rand de estadosIndices para que no se inicializen más de 1 agente por celda.
          rand = self.random.choice(estadosIndice)
          estados[rand] = 1
          estadosIndice.remove(rand)
              
        #Se pone el robot donde no hay algo más, estado es 6
        for i in range (5):
            rand = self.random.choice(estadosIndice)
            estados[rand] = 6
            estadosIndice.remove(rand)

        #Creamos el piso
        #Si el estado en indice cont es 6, entonces el valor se hace 0 porque no hay caja
        cont = 0
        contStacks = 0;
        StacksCompletos = int(K/5) + 1 
        for (content, x, y) in self.grid.coord_iter():
          estado = estados[cont]
          if estado == 6:
            estado = 0
            floor = floorAgent(cont, self, estado, False, True, False)
          elif estado == 1 and contStacks < StacksCompletos:
            #Al primer stack con estado 1 le asigno que es un residuo, los demás son estantes llenos
            if contStacks == 0:
              floor = floorAgent(cont, self, estado, True, False ,True)
            else:
              floor = floorAgent(cont, self, estado, True, False ,False)
            contStacks += 1
          else:
            floor = floorAgent(cont, self, estado, False, False, False)
          self.grid.place_agent(floor, (x,y))
          self.schedule.add(floor)
          cont += 1
          

        #Agregamos los robots donde el estado[] es 6
        #Checa cada celda, si el estado es 6, entonces agrega un robot ahí
        robots = 0
        for (content, x, y) in self.grid.coord_iter(): 
          if estados[robots] == 6:
            robot = robotAgent(robots+cont, self)
            self.grid.place_agent(robot, (x,y))
            self.schedule.add(robot)
          robots += 1
        
        # Aquí definimos con colector para obtener el grid completo. Aquí recompilamos la información.
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid})

    #Función que cuenta el número de celdas con sólo una caja
    def cajasSolas(model):
      contCajasSolas = 0
      grid = np.zeros((model.grid.width, model.grid.height))
      for cell in model.grid.coord_iter():
          cell_content, x, y = cell
          for content in cell_content:
            if isinstance(content, floorAgent):
              if content.estado == 1:
                contCajasSolas += 1
      return contCajasSolas

    def cajasStacks(model):
      contStacks = 0
      grid = np.zeros((model.grid.width, model.grid.height))
      for cell in model.grid.coord_iter():
          cell_content, x, y = cell
          for content in cell_content:
            if isinstance(content, floorAgent):
              if content.estado == 5:
                contStacks += 1
      return contStacks

    def step(self):
        '''
        En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
        '''
        self.datacollector.collect(self)
        self.schedule.step()
        

A continuación corremos el modelo

In [23]:
1# Definimos el tamaño del Grid
WIDTH = int(input('Width: '))
HEIGHT = int(input('Height: '))

# Definimos el número de cajas
K = int(input('Numero de cajas: '))

# Número de stacks completos
contStacksCompletos = int(K/5)

# Definimos el tiempo máximo en segundos
TIEMPO_MAX = float(input('Tiempo máximo de ejecución (segundos): '))

# Contador de movimientos realizados por los agentes
cont = 0

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
model = CleanWarehouseModel(WIDTH, HEIGHT, K)

#Corre si el tiempo transcurrido es menor al tiempo max, todavía no están los stacks llenos, y hay cajas solas.
while((time.time() - start_time) < TIEMPO_MAX and (model.cajasSolas() > 0 or model.cajasStacks() != contStacksCompletos)):
    model.step()
    cont += 1

# Tiempo necesario hasta que todas las cajas estén en pilas de 5 (o se haya llegado al tiempo máximo)
print('Tiempo de ordenamiento:', str(datetime.timedelta(seconds=(time.time() - start_time))))

# Número de movimientos realizados por todos los robots.
print('Número de movimientos realizados:', cont)

Width: 10
Height: 10
Numero de cajas: 33
Tiempo máximo de ejecución (segundos): 20
Tiempo de ordenamiento: 0:00:00.209915
Número de movimientos realizados: 654


Obtenemos la información que almacenó el colector, este nos entregará un DataFrame de pandas que contiene toda la información.

In [24]:
all_grid = model.datacollector.get_model_vars_dataframe()
print(all_grid.iloc[0][0])

[[1. 1. 1. 0. 1. 0. 1. 0. 0. 1.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 6. 0. 0. 0. 6. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 1. 1. 1.]
 [1. 0. 1. 0. 1. 0. 0. 1. 1. 0.]
 [0. 0. 1. 0. 0. 0. 1. 1. 0. 0.]
 [0. 6. 0. 6. 0. 0. 1. 0. 0. 0.]
 [0. 0. 6. 1. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 1. 1. 0. 0. 1. 0. 1.]]


Graficamos la información usando `matplotlib`

In [25]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap="Reds")

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [26]:
anim

## CUDA

El paso siguiente será modificar el modelo anterior para que funcione con CUDA

#Conclusiones


**Analysis**

Una estrategia que pudiera mejorar la eficiencia del programa, y como consequencia disminuir el tiempo dedicado y la cantidad de movimientos realizados por los robots es incrementar el número de cajas máximas por estante. Esto disminuiría el número de lugares (estantes) dónde los robots pueden dejar las cajas. En efecto, el tiempo dedicado y la cantidad de movimientos sería menor. Las siguientes simulaciones comprueban esta hipotesis. Como se puede observar, la simulación 1, que utiliza un programa donde el número máximo de cajas por estante es 5, tiene un tiempo de ordenamiento de 0.3 segundos y 1027 pasos. Mientras la simulación 2, que utiliza un programa donde el número máximo de cajas por estante es 10, tiene un tiempo de ordenamiento de 0.25 segundos y 764 pasos.


**Simulación 1**
* Width: 10
* Height: 10
* Numero de cajas: 33
* Tiempo máximo de ejecución (segundos): 20
* Tiempo de ordenamiento: 0:00:00.319362
* Número de movimientos realizados: 1027

**Simulación 2**
* Width: 10
* Height: 10
* Numero de cajas: 33
* Tiempo máximo de ejecución (segundos): 20
* Tiempo de ordenamiento: 0:00:00.248687
* Número de movimientos realizados: 764